In [1]:
import math
import numpy as np

# Hamming Codes

## Discussion

For a 16 bit data structure, 11 bits are data and 5 bits are redudant error checking:

|-|0*|1*|1|
|-|-|-|-|
|0*|0 |1 |1|
|1* |0 |0 |1|
|1 |0 |1 |0|

\* indicates this is a *parity bit*, a bit responsible for error checking (as opposed to a data bit)

The top left most box contains the extended parity bit. We'll get to that later, but for now we'll ignore it.

#### Parity Bits

A parity bit is responsible for counting the number of 1's in a specified section of the block and ensuring an even count of 1's. It changes its own value, 0 or 1, to ensure the count.

**Example**

Let's assume a 2x4 block, with the top-left most bit being our parity bit:

|?*|0|1|1|
|-|-|-|-|
|1|0|0|1|

There are 4 1's in the data, so our parity bit would assume the value of 0:

|0|0|1|1|
|-|-|-|-|
|1|0|0|1|

count of 1's: 4.


#### Error checking on a 16 bit block

Back to our 16 bit block, we have 4 active parity bits and then an *extended parity* bit in the top-left most block. For now, let's focus on the 4 primary parity bits:

|-|0*|1*|-|
|- |- |- |-|
|0*|- |- |-|
|1* |- |- |-|
|- |- |- |-|

*For this tutorial, rows and columns are 0-index based. So the left most column is column 0 and the rightmost column is column 3.*

##### Column Parities

The first column parity bit, in row 0 col 1, Checks all *odd columns* (0-indexed):

|-|X*|-|\*|
|-|- |-|- |
|-|\*|-|\*|
|-|\*|-|\*|
|-|\*|-|\*|

The second column parity bit looks to the right half of cols:

|-|- |X*|\*|
|-|- |-|-|
|-|- |\*|\*|
|-|- |\*|\*|
|-|- |\*|\*|

**Example**

|-|?\*|?\*|1|
|-|-|-|-|
|1|0 |1 |1|
|1|0 |0 |1|
|1|0 |1 |0|

Let's start with the first column parity bit, which checks columns 1 & 3:
- 1's in col 1: 0
- 1's in col 3: 3
- total: 3
- **parity bit: 1**

The second parity bit checks columns 2 & 3
- 1's in col 2: 2
- 1's in col 3: 3
- total: 5
- **parity bit: 1**

**Solution**

|-|1*|1*|1|
|-|-|-|-|
|1|0|1|1|
|1|0|0|1|
|1|0|1|0|

##### Row Parities

Similar to column parities, the first row parity (in the second row, we're skipping the first box) reviews all *odd index rows* while the second row parity reviews the bottom half of rows:

odd row parity

|-  |- |- |- |
|-  |- |- |- |
|?\*|\*|\*|\*|
|-  |- |- |- |
|\* |\*|\*|\*|

bottom half parity

|-|-|-|-|
|-|- |-|-|
|-|-|-|-|
|?*|\*|\*|\*|
|\*|\*|\*|\*|


**Example**

|-|0|1|1|
|-|-|-|-|
|?\*|0 |1 |1|
|?\*|0 |0 |1|
|1|0 |1 |0|

Starting with the odd-row parity, checking rows 1 & 3:
- 1's in row 1: 2
- 1's in row 3: 2
- total: 4
- **parity bit: 0**

The second half row parity checks rows 2 & 3:
- 1's in row 2: 1
- 1's in row 3: 2
- total: 3
**parity bit: 1**

**Solution**

|-|0|1|1|
|-|-|-|-|
|0|0|1|1|
|1|0|0|1|
|1|0|1|0|

## Implementation

In [2]:
def create_data_block(data):
    '''
    Given a 4x4 np array block, insert 1x11 data into data positions
    according to hamming code designations.
    '''
    block = np.zeros((4,4))
    data  = np.array(data)
    
    # ensure data is a single row with len 11
    if data.shape != (11,):
        raise Exception("Data must be a 1 x 11 array of 0's and 1's")
    # ensure data is all 0's and 1's:
    for item in data:
        if item not in (0, 1):
            raise Exception("Data must be a 1 x 11 array of 0's and 1's")
    
    # place data into block:
    block[0, -1] = data[0]
    block[1, 1:] = data[1:4]
    block[2, 1:] = data[4:7]
    block[3, :]  = data[7:]
    
    return block

In [3]:
def check_col_half_parity(block):
    n_cols = block.shape[1]
    count_ones = block[:, n_cols//2:].sum()
    return count_ones%2 == 0

def check_col_odd_parity(block):
    n_cols = block.shape[1]
    odd_col_idxs = np.arange(1, n_cols, 2)
    count_ones = block[:, odd_col_idxs].sum()
    return count_ones%2 == 0

def check_row_half_parity(block):
    n_rows = block.shape[0]
    count_ones = block[n_rows//2:, :].sum()
    return count_ones%2 == 0

def check_row_odd_parity(block):
    n_rows = block.shape[0]
    odd_row_idxs = np.arange(1, n_rows, 2)
    count_ones = block[odd_row_idxs, :].sum()
    return count_ones%2 == 0
    
def set_col_half_parity(block):
    parity_value =  int(not check_col_half_parity(block))
    block[0, 2] = parity_value
    return block

def set_col_odd_parity(block):
    parity_value = int(not check_col_odd_parity(block))
    block[0, 1] = parity_value
    return block

def set_row_half_parity(block):
    parity_value = int(not check_row_half_parity(block))
    block[2, 0] = parity_value
    return block

def set_row_odd_parity(block):
    parity_value = int(not check_row_odd_parity(block))
    block[1, 0] = parity_value
    return block


In [ ]:
def set_parities(block):
    set_col_half_parity(block)
    set_row_half_parity(block)
    set_col_odd_parity(block)
    set_row_odd_parity(block)
    return block

def check_parities(block, verbose=True):
    passed = True
    if not check_col_odd_parity(block):
        verbose and print('Column Odd Parity Violated')
        passed = False
    if not check_col_half_parity(block):
        verbose and print('Column Half Parity Violated')
        passed = False
    if not check_row_odd_parity(block):
        verbose and print('Row Odd Parity Violated')
        passed = False
    if not check_row_half_parity(block):
        verbose and print('Row Half Parity Violated')
        passed = False
    return passed

#### Example Usage

In [4]:
# Create Random data
data = np.around(np.random.rand(11))
# Store it in 4x4 data block
a = create_data_block(data)
print(a)
# Unless really lucky, our data block should have some parity issues
check_parities(a)
# Set parity bits
set_parities(a)
print('----')
# Now check for parity errors. None should be found
print(a)
f'Errors detected: {not check_parities(a)}'

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]
Column Odd Parity Violated
----
[[0. 1. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


'Errors detected: False'

#### Test

Let's test our parity engine on 1,000 cases. 

We'll create a random data block, set the parity bits and validate. We'll then flip a random data bit and check that parity has been broken.

In [5]:
def set_random_error(block):
    max_n = min(block.shape)
    rand_idx = np.random.randint(max_n-1, size=2)
    
    # Ignore the top left bit - we'll get to that one soon
    if rand_idx[0] == 0 and rand_idx[1] == 0:
        rand_idx = np.array([1,1])

    #flip value
    block[rand_idx[0], rand_idx[1]] = int(not block[rand_idx[0], rand_idx[1]])
    return block

In [18]:
trials = 100
trial_successes = 0
for _ in range(trials):
    block = create_data_block(np.around(np.random.rand(11)))
    set_parities(block)
    assert check_parities(block, verbose=False)
    set_random_error(block)
    assert not check_parities(block, verbose=False)
    trial_successes += 1

print(f'{trial_successes*100/trials:.0f}% trials succeeded')

100% trials succeeded
